<a href="https://colab.research.google.com/github/Code-sHimanshu/Collect-your-GamingTools/blob/main/Multilingual%20Chatbot%20for%20Sentiment%20and%20Emotion%20Analysis%20in%20text%20boxes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mount the drive


In [1]:
# prompt: mounnt drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [24]:
!pip install datasets
!pip install -q kaggle datasets


In [25]:
from google.colab import files
files.upload()  # Upload kaggle.json here


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"shimanshusingh","key":"670c7c2f4509804d7118734211518abd"}'}

In [26]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


**Load ISEAR Dataset (Kaggle CSV)**

In [30]:
!kaggle datasets download -d faisalsanto007/isear-dataset
!unzip -o isear-dataset.zip


Dataset URL: https://www.kaggle.com/datasets/faisalsanto007/isear-dataset
License(s): ODbL-1.0
isear-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  isear-dataset.zip
  inflating: eng_dataset.csv         


In [35]:
import os
print(os.listdir())


['.config', 'kaggle.json', 'drive', 'eng_dataset.csv', 'isear-dataset.zip', 'sample_data']


In [34]:
!unzip -o isear-dataset.zip


Archive:  isear-dataset.zip
  inflating: eng_dataset.csv         


In [36]:
import pandas as pd

# Load the ISEAR dataset
isear_df = pd.read_csv("eng_dataset.csv")

# Display first few rows to inspect
isear_df.head()


,ID,sentiment,content
0,10941,anger,At the point today where if someone says somet...
1,10942,anger,@CorningFootball IT'S GAME DAY!!!! T MIN...
2,10943,anger,This game has pissed me off more than any othe...
3,10944,anger,@spamvicious I've just found out it's Candice ...
4,10945,anger,@moocowward @mrsajhargreaves @Melly77 @GaryBar...


In [38]:
print(isear_df.columns)


Index(['ID', 'sentiment', 'content', 'source'], dtype='object')


In [40]:
# Rename columns to unify schema
isear_df = isear_df.rename(columns={
    'sentiment': 'label',
    'content': 'text'
})

# Drop 'ID' since it's not needed
isear_df = isear_df.drop(columns=['ID'])

# Keep only the relevant columns
isear_df = isear_df[['label', 'text', 'source']]

# Preview the cleaned data
isear_df.head()


,label,text,source
0,anger,At the point today where if someone says somet...,ISEAR
1,anger,@CorningFootball IT'S GAME DAY!!!! T MIN...,ISEAR
2,anger,This game has pissed me off more than any othe...,ISEAR
3,anger,@spamvicious I've just found out it's Candice ...,ISEAR
4,anger,@moocowward @mrsajhargreaves @Melly77 @GaryBar...,ISEAR


**Load GoEmotions Dataset (from HuggingFace)**

In [49]:
import pandas as pd

# Load ISEAR dataset
isear_df = pd.read_csv("eng_dataset.csv")

# Rename columns
isear_df.rename(columns={
    'sentiment': 'label',
    'content': 'text'
}, inplace=True)

# Add source column
isear_df['source'] = 'ISEAR'

# Ensure correct column order
isear_df = isear_df[['label', 'text', 'source']]

# Preview
isear_df.head()


,label,text,source
0,anger,At the point today where if someone says somet...,ISEAR
1,anger,@CorningFootball IT'S GAME DAY!!!! T MIN...,ISEAR
2,anger,This game has pissed me off more than any othe...,ISEAR
3,anger,@spamvicious I've just found out it's Candice ...,ISEAR
4,anger,@moocowward @mrsajhargreaves @Melly77 @GaryBar...,ISEAR


In [41]:
from datasets import load_dataset
import pandas as pd

# Load the train split of GoEmotions dataset
goemotions = load_dataset("go_emotions", split="train")

# Convert to DataFrame
goemotions_df = goemotions.to_pandas()

# Map numeric labels to emotion names
emotion_labels = goemotions.features['labels'].feature.names

# Expand multi-labels to single labels for simplicity (optional simplification)
goemotions_df['label'] = goemotions_df['labels'].apply(lambda x: emotion_labels[x[0]] if len(x) > 0 else 'neutral')
goemotions_df['text'] = goemotions_df['text']
goemotions_df['source'] = 'GoEmotions'

# Keep only required columns
goemotions_df = goemotions_df[['label', 'text', 'source']]

# Preview
goemotions_df.head()


,label,text,source
0,neutral,My favourite food is anything I didn't have to...,GoEmotions
1,neutral,"Now if he does off himself, everyone will thin...",GoEmotions
2,anger,WHY THE FUCK IS BAYLESS ISOING,GoEmotions
3,fear,To make her feel threatened,GoEmotions
4,annoyance,Dirty Southern Wankers,GoEmotions


**Load the Yelp Dataset**

In [42]:
# Load the Yelp dataset
yelp = load_dataset("yelp_review_full", split='train')
yelp_df = yelp.to_pandas()


**Map Star Ratings to Sentiment Labels**
*Yelp gives star ratings from 1 to 5.* We'll convert them to:

**1-2 stars → negative**

**3 stars → neutral**

**4-5 stars → positive**

In [43]:
# Convert ratings (0–4) to 1–5 scale and then to sentiment
def convert_rating_to_sentiment(rating):
    rating += 1  # Shift from 0-4 to 1-5
    if rating <= 2:
        return "negative"
    elif rating == 3:
        return "neutral"
    else:
        return "positive"

# Apply mapping
yelp_df['label'] = yelp_df['label'].apply(lambda x: convert_rating_to_sentiment(int(x)))
yelp_df['text'] = yelp_df['text']
yelp_df['source'] = 'Yelp'

# Select only the needed columns
yelp_df = yelp_df[['label', 'text', 'source']]

# Preview
yelp_df.head()


,label,text,source
0,positive,dr. goldberg offers everything i look for in a...,Yelp
1,negative,"Unfortunately, the frustration of being Dr. Go...",Yelp
2,positive,Been going to Dr. Goldberg for over 10 years. ...,Yelp
3,positive,Got a letter in the mail last week that said D...,Yelp
4,negative,I don't know what Dr. Goldberg was like before...,Yelp


In [44]:
print(yelp_df['label'].unique())


['positive' 'negative' 'neutral']


**Preprocess and combine the Datasets**

In [45]:
import pandas as pd
import re
from datasets import load_dataset


In [52]:
# Load Yelp
yelp = load_dataset("yelp_review_full", split="train")
yelp_df = yelp.to_pandas()

# Convert numeric label to sentiment
def convert_rating_to_sentiment(rating):
    if rating <= 2:
        return "negative"
    elif rating == 3:
        return "neutral"
    else:
        return "positive"

yelp_df['label'] = yelp_df['label'].apply(lambda x: convert_rating_to_sentiment(int(x) + 1))
yelp_df['text'] = yelp_df['text']
yelp_df['source'] = 'Yelp'
yelp_df = yelp_df[['label', 'text', 'source']]


In [51]:
combined_df = pd.concat([isear_df, goemotions_df, yelp_df], ignore_index=True)
combined_df = combined_df.sample(frac=1).reset_index(drop=True)

# Preview
print(combined_df['source'].value_counts())  # Should show all three sources
combined_df.head()


source
Yelp          650000
GoEmotions     43410
ISEAR           7102
Name: count, dtype: int64


,label,text,source
0,positive,Cafe Zoma is great! I love that most people co...,Yelp
1,positive,Had dinner with a largeish group on a Friday n...,Yelp
2,neutral,I have a love/hate relationship with this fast...,Yelp
3,positive,Quantity and quality.\n\nI enjoy the pastrami ...,Yelp
4,positive,If you have a higher end grill that is basical...,Yelp


**Preview few samples from each dataset**

In [53]:
# Show some ISEAR samples
print("ISEAR samples:\n", combined_df[combined_df['source'] == 'ISEAR'].sample(3))

# Show some GoEmotions samples
print("\nGoEmotions samples:\n", combined_df[combined_df['source'] == 'GoEmotions'].sample(3))

# Show some Yelp samples
print("\nYelp samples:\n", combined_df[combined_df['source'] == 'Yelp'].sample(3))


ISEAR samples:
           label                                               text source
74284   sadness  @shopgreenwich #ldf16 what shall we do this we...  ISEAR
559306      joy  Chris would take full responsibility and would...  ISEAR
297271      joy         gifs on iOS10 messaging app are hilarious.  ISEAR

GoEmotions samples:
        label                                               text      source
205509     6  She actually survived the incident, I don’t kn...  GoEmotions
172037    20         I'm so sorry! I hope you are doing better!  GoEmotions
433392    27  Speech should be illegal if it is an incitemen...  GoEmotions

Yelp samples:
            label                                               text source
225704   neutral  Your average step-above-Waffle-House type of p...   Yelp
192023  positive  I work close by and have eaten at Las Glorias ...   Yelp
78812   negative  \"I'm STARVING! waited 34 mins AFTER gettin se...   Yelp


**Clean the Dataset**

In [54]:
import re

def clean_text(text):
    text = text.lower()  # Lowercase
    text = re.sub(r'\n', ' ', text)  # Replace newlines with space
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove special characters
    return text.strip()

# Apply cleaning
combined_df['text'] = combined_df['text'].apply(clean_text)

# Preview cleaned text
combined_df.head()


,label,text,source
0,positive,cafe zoma is great i love that most people com...,Yelp
1,positive,had dinner with a largeish group on a friday n...,Yelp
2,neutral,i have a lovehate relationship with this fast ...,Yelp
3,positive,quantity and qualitynni enjoy the pastrami and...,Yelp
4,positive,if you have a higher end grill that is basical...,Yelp


**Encode the labels for ML.**

In [55]:
from sklearn.preprocessing import LabelEncoder

# Initialize label encoder
label_encoder = LabelEncoder()

# Fit and transform labels
combined_df['label'] = label_encoder.fit_transform(combined_df['label'])

# Save label mapping for reference
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label Mapping:", label_mapping)

# Preview dataset with encoded labels
combined_df.head()


Label Mapping: {'0': np.int64(0), '1': np.int64(1), '10': np.int64(2), '11': np.int64(3), '12': np.int64(4), '13': np.int64(5), '14': np.int64(6), '15': np.int64(7), '16': np.int64(8), '17': np.int64(9), '18': np.int64(10), '19': np.int64(11), '2': np.int64(12), '20': np.int64(13), '21': np.int64(14), '22': np.int64(15), '23': np.int64(16), '24': np.int64(17), '25': np.int64(18), '26': np.int64(19), '27': np.int64(20), '3': np.int64(21), '4': np.int64(22), '5': np.int64(23), '6': np.int64(24), '7': np.int64(25), '8': np.int64(26), '9': np.int64(27), 'anger': np.int64(28), 'fear': np.int64(29), 'joy': np.int64(30), 'negative': np.int64(31), 'neutral': np.int64(32), 'positive': np.int64(33), 'sadness': np.int64(34)}


,label,text,source
0,33,cafe zoma is great i love that most people com...,Yelp
1,33,had dinner with a largeish group on a friday n...,Yelp
2,32,i have a lovehate relationship with this fast ...,Yelp
3,33,quantity and qualitynni enjoy the pastrami and...,Yelp
4,33,if you have a higher end grill that is basical...,Yelp


**Save the Combined Dataset as a CSV File**

In [56]:
# Save the dataset as a CSV file
combined_df.to_csv("combined_emotion_dataset.csv", index=False)

print("Dataset saved successfully as 'combined_emotion_dataset.csv'!")


Dataset saved successfully as 'combined_emotion_dataset.csv'!


In [58]:
from google.colab import files  # Use this if running on Google Colab

# Save the dataset as a CSV file
csv_filename = "combined_emotion_dataset.csv"
combined_df.to_csv(csv_filename, index=False)

# Download the file
files.download(csv_filename)  # This will prompt a download


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Split the Dataset into Train, Test, and Validation Sets**

In [59]:
from sklearn.model_selection import train_test_split

# Split dataset into train (70%), test (15%), and validation (15%)
train_texts, temp_texts, train_labels, temp_labels = train_test_split(
    combined_df["text"], combined_df["label"], test_size=0.3, random_state=42, stratify=combined_df["label"]
)

val_texts, test_texts, val_labels, test_labels = train_test_split(
    temp_texts, temp_labels, test_size=0.5, random_state=42, stratify=temp_labels
)

# Print dataset sizes
print(f"Training set: {len(train_texts)} samples")
print(f"Validation set: {len(val_texts)} samples")
print(f"Test set: {len(test_texts)} samples")


Training set: 490358 samples
Validation set: 105077 samples
Test set: 105077 samples


**Build a Classification Model**

Now, we will build a basic text classification model using TF-IDF for text representation and a Logistic Regression model for classification.

In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Convert text into TF-IDF features
vectorizer = TfidfVectorizer(max_features=5000)  # Limit vocabulary to 5000 words
X_train = vectorizer.fit_transform(train_texts)
X_val = vectorizer.transform(val_texts)
X_test = vectorizer.transform(test_texts)

# Train Logistic Regression model
model = LogisticRegression(max_iter=500)
model.fit(X_train, train_labels)

# Evaluate on validation set
val_preds = model.predict(X_val)
val_accuracy = accuracy_score(val_labels, val_preds)
print(f"Validation Accuracy: {val_accuracy:.4f}")

# Display classification report
print("Validation Classification Report:\n", classification_report(val_labels, val_preds))


Validation Accuracy: 0.7460
Validation Classification Report:
               precision    recall  f1-score   support

           0       0.60      0.40      0.48       620
           1       0.66      0.65      0.66       337
           2       0.42      0.09      0.15       248
           3       0.41      0.10      0.17        87
           4       0.00      0.00      0.00        37
           5       0.30      0.07      0.11       105
           6       0.59      0.17      0.27        76
           7       0.74      0.76      0.75       314
           8       0.00      0.00      0.00        10
           9       0.44      0.22      0.30       152
          10       0.61      0.45      0.52       230
          11       0.00      0.00      0.00        16
          12       0.50      0.33      0.40       232
          13       0.45      0.37      0.41       146
          14       0.00      0.00      0.00         9
          15       0.36      0.05      0.08       104
          16      

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


**Train XGBoost Classifier**

**Save the Logistic Regression model using joblib**

In [68]:
import pickle

# Save model
with open("logistic_regression_model.pkl", "wb") as file:
    pickle.dump(model, file)

# Save vectorizer
with open("tfidf_vectorizer.pkl", "wb") as file:
    pickle.dump(vectorizer, file)

print("Model and vectorizer saved successfully.")


Model and vectorizer saved successfully.


**Download Model**

In [69]:
from google.colab import files

# Download the model and vectorizer
files.download("logistic_regression_model.pkl")
files.download("tfidf_vectorizer.pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**BiLSTM model**

Tokenize the text



**Split Your Combined Data**

In [72]:
from sklearn.model_selection import train_test_split

# Assuming combined_df contains 'text' and 'label' columns
train_df, temp_df = train_test_split(combined_df, test_size=0.2, stratify=combined_df['label'], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['label'], random_state=42)

# Now extract the text and labels
X_train = train_df['text'].astype(str).tolist()
y_train = train_df['label'].tolist()

X_val = val_df['text'].astype(str).tolist()
y_val = val_df['label'].tolist()

X_test = test_df['text'].astype(str).tolist()
y_test = test_df['label'].tolist()


**Tokenization and Padding**

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Set vocabulary size and max sequence length
vocab_size = 20000  # You can adjust this depending on memory
max_length = 100    # Max tokens per sequence (can be tuned)
trunc_type = 'post'
padding_type = 'post'

# Tokenize text
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

# Convert text to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_val)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences
X_train_pad = pad_sequences(X_train_seq, maxlen=max_length, padding=padding_type, truncating=trunc_type)
X_val_pad = pad_sequences(X_val_seq, maxlen=max_length, padding=padding_type, truncating=trunc_type)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_length, padding=padding_type, truncating=trunc_type)


NameError: name 'X_train' is not defined

**Label Encoding**

In [74]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Initialize encoder
label_encoder = LabelEncoder()

# Fit and transform labels
y_train_enc = label_encoder.fit_transform(y_train)
y_val_enc = label_encoder.transform(y_val)
y_test_enc = label_encoder.transform(y_test)

# Convert to one-hot
y_train_cat = to_categorical(y_train_enc)
y_val_cat = to_categorical(y_val_enc)
y_test_cat = to_categorical(y_test_enc)

# Number of unique emotion classes
num_classes = y_train_cat.shape[1]
print("Number of Classes:", num_classes)


Number of Classes: 35


In [2]:
# Tokenizer must be fitted before setting vocab_size
vocab_size = len(tokenizer.word_index) + 1  # Add 1 for padding token


NameError: name 'tokenizer' is not defined

**Build & Train BiLSTM Model**

**verify the input dimensions**

In [76]:
print(f"Vocab Size: {vocab_size}")
print(f"Max Sequence Length: {max_len}")
print(f"Number of Classes: {num_classes}")
print(f"X_train_pad Shape: {X_train_pad.shape}")
print(f"y_train_cat Shape: {y_train_cat.shape}")


Vocab Size: 20000
Max Sequence Length: 100
Number of Classes: 35
X_train_pad Shape: (560409, 100)
y_train_cat Shape: (560409, 35)


In [79]:
Embedding(input_dim=vocab_size, output_dim=128, input_length=max_len)


<Embedding name=embedding_3, built=False>

**Define the BiLSTM model**

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional

# Define the model
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=128, input_length=max_len),
    Bidirectional(LSTM(64, return_sequences=True)),
    Dropout(0.4),
    Bidirectional(LSTM(64)),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.build(input_shape=(None, max_len))  # Force model to build

# Display model summary
model.summary()


NameError: name 'vocab_size' is not defined

**Introduce Early Stopping to stop over fitting**

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32, callbacks=[early_stopping])


**Model Training**

In [81]:
history = model.fit(X_train_pad, y_train_cat,
                    validation_data=(X_val_pad, y_val_cat),
                    epochs=10, batch_size=32)


Epoch 1/10
17513/17513 ━━━━━━━━━━━━━━━━━━━━ 382s 22ms/step - accuracy: 0.6220 - loss: 0.9681 - val_accuracy: 0.7432 - val_loss: 0.6822
Epoch 2/10
17513/17513 ━━━━━━━━━━━━━━━━━━━━ 377s 22ms/step - accuracy: 0.7531 - loss: 0.6688 - val_accuracy: 0.7548 - val_loss: 0.6448
Epoch 3/10
17513/17513 ━━━━━━━━━━━━━━━━━━━━ 382s 22ms/step - accuracy: 0.7732 - loss: 0.6119 - val_accuracy: 0.7567 - val_loss: 0.6455
Epoch 4/10
17513/17513 ━━━━━━━━━━━━━━━━━━━━ 382s 22ms/step - accuracy: 0.7928 - loss: 0.5658 - val_accuracy: 0.7589 - val_loss: 0.6525
Epoch 5/10
17513/17513 ━━━━━━━━━━━━━━━━━━━━ 383s 22ms/step - accuracy: 0.8091 - loss: 0.5263 - val_accuracy: 0.7532 - val_loss: 0.6779
Epoch 6/10
17513/17513 ━━━━━━━━━━━━━━━━━━━━ 384s 22ms/step - accuracy: 0.8255 - loss: 0.4849 - val_accuracy: 0.7482 - val_loss: 0.7258
Epoch 7/10
17513/17513 ━━━━━━━━━━━━━━━━━━━━ 388s 22ms/step - accuracy: 0.8412 - loss: 0.4506 - val_accuracy: 0.7453 - val_loss: 0.7717
Epoch 8/10
17513/17513 ━━━━━━━━━━━━━━━━━━━━ 434s 22ms/s

In [77]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout

# Model architecture
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=128, input_length=max_len),
    Bidirectional(LSTM(64, return_sequences=True)),
    Dropout(0.5),
    Bidirectional(LSTM(32)),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Summary
model.summary()

# Train model
history = model.fit(
    X_train_pad, y_train_cat,
    validation_data=(X_val_pad, y_val_cat),
    epochs=5,
    batch_size=128
)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_2 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_3 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
  23/4379 ━━━━━━━━━━━━━━━━━━━━ 2:33 35ms/step - accuracy: 0.2972 - loss: 3.1649

KeyboardInterrupt: 